In [7]:
import pandas as pd

In [8]:
# tx = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/LEUVEN cohort_AS.xlsx')
hla_p = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/Original/HR typing patient.xlsx')
hla_d = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/Original/HR typing donor.xlsx')

In [37]:
patient_hla_cols = ['Rec1stA', 'Rec2ndA', 'Rec1stB', 'Rec2ndB', 'Rec1stC', 'Rec2ndC', '1stDRB', '2ndDRB', '1stDRW', '2ndDRW', '1stDQB', '2ndDQB', '1stDQA', '2ndDQA']
cols = ['TX number', 'Rec ID'] + patient_hla_cols
hla_patient = hla_p[cols].assign(
    patient_hla_typing = hla_p[patient_hla_cols].apply(lambda x: ', '.join(set([hla for hla in x])), axis=1)
).drop(patient_hla_cols, axis=1)
hla_patient

,TX number,Rec ID,patient_hla_typing
0,2725,84859230,"DRB3*02:02, B*18:01, DQA1*05:05, B*18:03, DRB1..."
1,2726,71420038,"B*44:02, DRB1*11:01, DRB3*02:02, DQA1*05:05, D..."
2,2727,71612089,"A*23:01, B*44:03, DRB3*01:01, DQB1*02:01, DQB1..."
3,2728,74510751,"A*24:02, C*16:01, DRB3*02:02, B*44:03, DQA1*05..."
4,2729,83270124,"C*06:02, DRB3*01:01, DQB1*02:01, DRB1*15:01, A..."
...,...,...,...
921,3857,66275884,"DRB1*11:01, A*24:02, B*13:02, DRB3*02:02, C*06..."
922,3858,66646548,"A*32:01, DRB1*13:03, DRB3*01:01, DQB1*02:02, D..."
923,3859,64448509,"DQB1*06:04, DRB1*13:02, A*03:01, C*04:01, A*68..."
924,3860,65415259,"DRB1*11:01, DRB3*02:02, DRB3*01:01, DQA1*05:05..."


In [38]:
donor_hla_cols = ['D1stA', 'D2ndA', 'D1stB', 'D2ndB', 'D1stC', 'D2ndC','1stDRB', '2ndDRB', '1stDRW', '2ndDRW', '1stDQB', '2ndDQB', '1stDQA', '2ndDQA']
cols = ['TX number', 'Don ET'] + donor_hla_cols
hla_donor = hla_d[cols].assign(
    donor_hla_typing = hla_d[donor_hla_cols].apply(lambda x: ', '.join(set([hla for hla in x])), axis=1)
).drop(donor_hla_cols, axis=1)
hla_donor

,TX number,Don ET,donor_hla_typing
0,2725,98468,"DQB1*03:03, DRB3*02:02, DQA1*05:05, DRB1*11:04..."
1,2726,98494,"B*44:02, DRB1*11:01, DRB3*02:02, DQA1*05:05, D..."
2,2727,98536,"A*23:01, B*44:03, DRB3*01:01, DQB1*02:01, DQB1..."
3,2728,98536,"A*23:01, B*44:03, DRB3*01:01, DQB1*02:01, DQB1..."
4,2729,98671,"DRB3*01:01, DQB1*02:01, DRB1*04:01, A*03:01, B..."
...,...,...,...
921,3857,147280,"DQB1*06:04, DRB1*13:02, C*15:02, DQB1*02:02, A..."
922,3858,147280,"DQB1*06:04, DRB1*13:02, C*15:02, DQB1*02:02, A..."
923,3859,147279,"DRB3*02:02, C*15:02, DRB1*04:01, A*03:01, B*51..."
924,3860,147302,"A*32:01, B*44:02, DRB3*02:02, DQA1*05:05, B*38..."


In [39]:
hlas = hla_patient.merge(hla_donor, on='TX number')
hlas.to_csv('/Users/Danial/UMCUtrecht/Leuven/Processed/HLA Typings.csv')

In [139]:
from src.leuven.utility.dataclasses import (
    Transplant, 
    Participant,
    Luminex,
)
from src.leuven.antibody_analysis import AntibodyAnalysis
%load_ext autoreload
%autoreload 2

df_lsa = pd.read_csv('~/UMCUtrecht/Leuven/Processed/lsa_joined.csv')
# txs = []
ids = [2767, 2797, 2801, 2817, 2835, 2838, 2841, 2860, 2867, 2872, 2876, 2885, 2896, 2907, 2963, 2974, 2982, 2990, 2992, 3002, 3003, 3006,
       3026, 3042, 3075, 3104, 3105, 3111, 3118, 3135, 3140, 3152, 3179, 3188, 3211, 3218, 3229, 3265, 3272, 3279, 3288, 3295, 3296, 3312,
       3353, 3371, 3406, 3420, 3421, 3423, 3428, 3451, 3452, 3453, 3456, 3463, 3466, 3478, 3481, 3490, 3502, 3520, 3524, 3529, 3535, 3546,
       3560, 3565, 3591, 3594, 3607, 3608, 3622, 3626, 3656, 3665, 3695, 3700, 3703, 3705, 3730, 3731, 3742, 3765, 3775, 3784, 3793, 3797,
       3798, 3802, 3811, 3824, 3845, 3850, 3861]

results = []
pipeline = AntibodyAnalysis()
hlas_dsa = hlas[hlas['TX number'].apply(lambda x: x in ids)]
for row in hlas_dsa.iterrows():
    tx_id, patient_id, patient_hla, donor_id, donor_hla = row[1]
    lsa = [Luminex(hla,assign) for assign, hla in df_lsa[df_lsa.tx_id == tx_id][['assignment', 'allele']].values]
    tx = Transplant(
        id=tx_id, 
        donor=Participant(donor_hla), 
        recipient=Participant(patient_hla),
        lsa=lsa,
    )
    pipeline.find_desa(tx)
    results.append(pipeline.export_results())

res = pd.DataFrame(results)
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
# res.LuminexBeads.values

In [135]:
res.Donor_HLA.values[0].hlas

[HLA(high_res='DRB1*16:01'),
 HLA(high_res='DRB1*15:01'),
 HLA(high_res='A*03:01'),
 HLA(high_res='B*07:02'),
 HLA(high_res='DQB1*06:02'),
 HLA(high_res='C*02:02'),
 HLA(high_res='C*07:02'),
 HLA(high_res='DRB5*02:02'),
 HLA(high_res='DQB1*05:02'),
 HLA(high_res='A*11:01'),
 HLA(high_res='DQA1*01:02'),
 HLA(high_res='B*27:02'),
 HLA(high_res='DRB5*01:05')]

In [142]:
res.DESA_info.apply(lambda x: 1 if len(x) else 0)

,Epitope_Mismatch,Tx_id,Donor_HLA,Recipient_HLA,LuminexBeads,DESA_Status,DESA_info
0,"{'51R': 'DRB3*02:02', '30H[DR]': 'DRB3*02:02',...",2767,"Participant(hlas=[HLA(high_res='A*24:02'), HLA...","Participant(hlas=[HLA(high_res='A*24:02'), HLA...","{'Positive': {'Specificity': {'B*44:02', 'A*23...",No DESA,()
1,"{'32L': 'B*44:02', '71TN': 'B*44:02', '166ES':...",2797,"Participant(hlas=[HLA(high_res='B*44:02'), HLA...","Participant(hlas=[HLA(high_res='DRB3*01:01'), ...",{'non-Positive': {'Specificity': {'DRB1*15:02'...,DESA,"((32L, B*44:02), (166ES, B*44:02), (24T, B*44:..."
2,"{'57DE': 'DRB1*11:04', '32Y': 'DRB1*11:04', '1...",2801,"Participant(hlas=[HLA(high_res='DRB3*02:02'), ...","Participant(hlas=[HLA(high_res='A*24:02'), HLA...","{'non-Positive': {'Specificity': {'A*66:02', '...",No DESA,()
3,"{'156QA': 'A*24:02', '144KR': 'A*24:02', '138M...",2817,"Participant(hlas=[HLA(high_res='DRB1*01:01'), ...","Participant(hlas=[HLA(high_res='DRB1*01:01'), ...","{'Positive': {'Specificity': {'B*44:03', 'B*38...",DESA,"((166DG, A*24:02))"
4,"{'9F[A]': 'A*32:01', '80I': 'A*32:01', '76ESI'...",2835,"Participant(hlas=[HLA(high_res='A*32:01'), HLA...","Participant(hlas=[HLA(high_res='C*16:01'), HLA...","{'non-Positive': {'Specificity': {'B*44:02', '...",DESA,"((76ESI, A*32:01), (90D, C*06:02), (80K, C*06:..."
...,...,...,...,...,...,...,...
90,"{'156DA': 'B*44:02', '51R': 'DRB3*02:02', '30H...",3811,"Participant(hlas=[HLA(high_res='B*44:02'), HLA...","Participant(hlas=[HLA(high_res='A*23:01'), HLA...","{'Positive': {'Specificity': {' DPB1*13:01', '...",No DESA,()
91,"{'21H': 'C*15:02', '116L': 'C*15:02', '113HD':...",3824,"Participant(hlas=[HLA(high_res='C*15:02'), HLA...","Participant(hlas=[HLA(high_res='B*44:02'), HLA...","{'Positive': {'Specificity': {'A*68:02', 'A*69...",No DESA,()
92,"{'13FE': 'DRB1*01:01', '70QA': 'DRB1*01:01', '...",3845,"Participant(hlas=[HLA(high_res='DRB1*01:01'), ...","Participant(hlas=[HLA(high_res='DRB3*02:02'), ...",{'non-Positive': {'Specificity': {'DRB1*15:02'...,No HLAE-Abs,()
93,"{'13FE': 'DRB1*01:01', '30C': 'DRB1*01:01', '3...",3850,"Participant(hlas=[HLA(high_res='DRB1*01:01'), ...","Participant(hlas=[HLA(high_res='DRB3*01:01'), ...","{'Positive': {'Specificity': {'DQB1*04:02', 'D...",DESA,"((52SK, DQA1*01:01), (129QS, DQA1*01:01), (116..."
